In [1]:
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pandas as pd

from preprocessing.generate_negatives import gen_neg_sample

In [5]:
test_edges = pd.read_parquet('data/test_edges.parquet')
test_features = pd.read_parquet('data/test_features.parquet')
test_labels = pd.read_parquet('data/test_labels.parquet')

val_edges = pd.read_parquet('data/val_edges.parquet')
val_features = pd.read_parquet('data/val_features.parquet')
val_labels = pd.read_parquet('data/val_labels.parquet')

train_edges = pd.read_parquet('data/train_edges.parquet')
train_features = pd.read_parquet('data/train_features.parquet')
train_labels = pd.read_parquet('data/train_labels.parquet')

In [7]:
train_labels_neg = gen_neg_sample(train_edges, train_labels)
val_labels_neg = gen_neg_sample(val_edges, val_labels)
test_labels_neg = gen_neg_sample(test_edges, test_labels)

test_edges.shape:  (8675721, 3)
test_labels.shape:  (330526, 3)
hist_purchases.shape:  (9006247, 2)  - Is total:  True
test_edges.shape:  (9006247, 3)
test_labels.shape:  (367218, 3)
hist_purchases.shape:  (9373465, 2)  - Is total:  True
test_edges.shape:  (9337086, 3)
test_labels.shape:  (328203, 3)
hist_purchases.shape:  (9665289, 2)  - Is total:  True


In [10]:
train_labels_neg.to_csv("data/train_labels_neg.csv", index=None)
val_labels_neg.to_csv("data/val_labels_neg.csv", index=None)
test_labels_neg.to_csv("data/test_labels_neg.csv", index=None)

In [7]:
class BipartiteData(Data):
    def __init__(self, edge_index=None, x_s=None, x_t=None):
        super().__init__()
        self.edge_index = edge_index
        self.x_s = x_s
        self.x_t = x_t
    
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index':
            return torch.tensor([[self.x_s.size(0)], [self.x_t.size(0)]])
        else:
            return super().__inc__(key, value, *args, **kwargs)

In [12]:
edge_index = test_edges.loc[:, ["user_id", "product_id"]].to_numpy().T

In [13]:
data = BipartiteData(edge_index)